# Installazione delle librerie
Visto che colab e' un ambiente di runtime provvisorio, eseguire questo script per ogni prima instanza della macchina, cosi da settare l'ambiente prima dell'esecuzione.

In [ ]:
# .\.venv\Scripts\activate ---> for set local environment

! pip install pdfplumber
! pip install transformers
! pip install PyPDF2
! pip install spacy

! pip install torch
! pip install torchvision 
! pip install torchaudio

! pip install quart

# ! pip install https://huggingface.co/bullmount/it_nerIta_trf/resolve/main/it_nerIta_trf-any-py3-none-any.whl
! python -m spacy download it_core_news_lg
! python -m spacy download it_core_news_sm

In [ ]:
!pip freeze >> requirements.txt

In [ ]:
! pip install Quart-Schema
! pip install azure-storage-blob

In [ ]:
# spiegazione di come funziona https://www.youtube.com/watch?v=fgmA9am9qCo&ab_channel=ASLearning

import spacy
from PyPDF2 import PdfReader
from transformers import BartTokenizer, BartForConditionalGeneration
from spacy import displacy


percorso_file_pdf = './dispense-neuroetica-ed-etica-applicata-mfil03-1-.pdf'
# percorso_file_pdf = 'digitalinnovation.pdf'

model = BartForConditionalGeneration.from_pretrained('facebook/bart-large-cnn')
tokenizer = BartTokenizer.from_pretrained('facebook/bart-large-cnn')

# Loading spacy models
nlp = spacy.load("it_core_news_lg")
# nlpNer = spacy.load("it_nerIta_trf")

In [ ]:
#Used model it_core_news_lg
#SPLIT INFO PARAGRAPH

# Read PDF Files
with open(percorso_file_pdf, "rb") as file:
    pdf_reader = PdfReader(file)
    text = ""
    for page_num in range(len(pdf_reader.pages)):
        page = pdf_reader.pages[page_num]
        text += page.extract_text()
doc = nlp(text)

paragrafi = []
paragrafo_corrente = ""

for sent in doc.sents:
    paragrafo_corrente += sent.text

    if len(paragrafo_corrente) >= 200:
        paragrafi.append(paragrafo_corrente)
        paragrafo_corrente = ""

if paragrafo_corrente:
    paragrafi.append(paragrafo_corrente)


In [ ]:
#Used model it_nerIta_trf

#NER for page MODEL1
with open(percorso_file_pdf, "rb") as file:
    pdf_reader = PdfReader(file)
    text = ""
    num = 0
    # Estrai il testo da ciascuna pagina del PDF
    for page_num in range(len(pdf_reader.pages)):
        page = pdf_reader.pages[page_num]
        print('--------------------------------------PAGE-----------------------------------------', num)
        pageText = page.extract_text()
        doc = nlpNer(pageText)
        displacy.render(doc, style="ent",jupyter=True)
        num = num + 1

In [ ]:
#Used model it_nerIta_trf

#NER for page MODEL1
with open("File test.docx", "rb") as file:
    pdf_reader = PdfReader(file)
    text = ""
    num = 0
    # Estrai il testo da ciascuna pagina del PDF
    for page_num in range(len(pdf_reader.pages)):
        page = pdf_reader.pages[page_num]
        print('--------------------------------------PAGE-----------------------------------------', num)
        pageText = page.extract_text()

In [ ]:
#Used model it_nerIta_trf

#NER for page MODEL1
with open(percorso_file_pdf, "rb") as file:
    pdf_reader = PdfReader(file)
    text = ""
    num = 0
    # Estrai il testo da ciascuna pagina del PDF
    for page_num in range(len(pdf_reader.pages)):
        page = pdf_reader.pages[page_num]
        print('--------------------------------------PAGE-----------------------------------------', num)
        pageText = page.extract_text()
        doc = nlp(pageText)
        displacy.render(doc, style="ent",jupyter=True)
        num = num + 1

In [ ]:
doc = nlp('Apple è un azienda di informatica')
print('Entità:', doc.ents[0].ents[0].label_)

for ent in doc.ents:
        print(ent.end)
        print(ent.ents[0].label_)
        print(ent.start)
        print(ent.text)

In [ ]:
#SUMMARIZATION for paragraph
#For finetuning https://towardsdatascience.com/fine-tuning-the-bart-large-model-for-text-summarization-3c69e4c04582

for paragraph in paragrafi:
  inputs = tokenizer([paragraph], return_tensors='pt')
  summary_ids = model.generate(inputs['input_ids'], max_length=100, early_stopping=False)
  print([tokenizer.decode(g, skip_special_tokens=True) for g in summary_ids])

In [ ]:
#SUMMARIZATION for page
token_limit_model = 1030

with open(percorso_file_pdf, "rb") as file:
    pdf_reader = PdfReader(file)
    text = ""
    num = 1
    # Estrai il testo da ciascuna pagina del PDF
    for page_num in range(len(pdf_reader.pages)):
        page = pdf_reader.pages[page_num]
        print('--------------------------------------PAGE-----------------------------------------', num)
        num = num + 1
        pageText = page.extract_text()
        inputs = tokenizer([pageText], return_tensors='pt')
        token_inside_page = len(inputs['input_ids'][0])
        print("token_inside_page", token_inside_page)

        if token_inside_page >= token_limit_model:
            continue

        summary_ids = model.generate(inputs['input_ids'], max_length=100, early_stopping=False)
        print([tokenizer.decode(g, skip_special_tokens=True) for g in summary_ids])


In [ ]:
#